In [3]:
#loop through and upload all csvs

import pandas as pd
import os
import glob
import numpy as np
import requests
import time

filenames = glob.glob("Big-Data-Bowl/Data/tracking_gameId_2017******.csv")
tracking = []
tracking = [pd.read_csv(filename,encoding='cp1252') for filename in filenames]
games = pd.read_csv("Big-Data-Bowl/Data/games.csv")
players = pd.read_csv("Big-Data-Bowl/Data/players.csv")
plays = pd.read_csv("Big-Data-Bowl/Data/plays.csv")

In [4]:
## want to find plays where the pass length is > 20 yards
deep_pass = plays[plays['PassLength']>20]
deep_passsort = deep_pass.sort_values(by='gameId',ascending=True) 

In [9]:
#print(len(deep_pass))

635


In [5]:
## want to find the gameindex for each of the deep passes so we can easily sort through our list of dfs
indices = []
play_index = []
for i in range(len(deep_passsort)):
    for j in range(len(tracking)):
        if deep_passsort.iloc[i,0] == tracking[j].iloc[j,-2]:
            indices.append(j)
            play_index.append(deep_passsort.iloc[i,1])

deep_passsort.insert(0,'DF_index',indices)
deep_pass_track = tracking.copy()
for i in range(len(tracking)):
    temp = deep_passsort[deep_passsort['DF_index'] == i] #for each game, bring up the deep plays
    temp_plays = temp['playId']
    deep_pass_track[i] = tracking[i][tracking[i]['playId'].isin(temp_plays)]

In [6]:
## now we have all the player tracking data for each big throw for all the players
## need to filter out for only the relevant players ie Safeties
safety = ['SS','FS']
safeties = players[players['PositionAbbr'].isin(safety)]
print(safeties.iloc[2,0])


79897


In [11]:
## keep only the tracking data of the safeties
safety_track = deep_pass_track.copy()
for i in range(len(deep_pass_track)):
    safety_track[i] = deep_pass_track[i][deep_pass_track[i]['nflId'].isin(safeties['nflId'])]

safety_track_comb = pd.concat(safety_track)
#safety_track_comb.to_csv('Safety Tracking on Deep Throws')

#safety_track_comb.tail(10)

,time,x,y,s,dis,dir,event,nflId,displayName,jerseyNumber,team,frame.id,gameId,playId
235634,2017-10-17T03:09:05Z,108.80,1.03,7.30,0.74,120.05,NaN,2558060.0,Malik Hooker,29.0,away,96,2017101600,3277
235635,2017-10-17T03:09:05Z,109.45,0.66,7.09,0.74,119.59,NaN,2558060.0,Malik Hooker,29.0,away,97,2017101600,3277
235636,2017-10-17T03:09:05Z,110.07,0.30,6.85,0.72,119.12,NaN,2558060.0,Malik Hooker,29.0,away,98,2017101600,3277
235637,2017-10-17T03:09:05Z,110.68,-0.04,6.58,0.70,118.67,NaN,2558060.0,Malik Hooker,29.0,away,99,2017101600,3277
235638,2017-10-17T03:09:05Z,111.26,-0.35,6.33,0.65,118.22,NaN,2558060.0,Malik Hooker,29.0,away,100,2017101600,3277
235639,2017-10-17T03:09:09Z,115.12,-0.49,1.58,0.17,313.29,NaN,2558060.0,Malik Hooker,29.0,away,101,2017101600,3277
235640,2017-10-17T03:09:09Z,114.99,-0.37,1.58,0.17,312.96,NaN,2558060.0,Malik Hooker,29.0,away,102,2017101600,3277
235641,2017-10-17T03:09:09Z,114.87,-0.26,1.59,0.16,312.86,NaN,2558060.0,Malik Hooker,29.0,away,103,2017101600,3277
235642,2017-10-17T03:09:09Z,114.76,-0.15,1.59,0.16,312.74,NaN,2558060.0,Malik Hooker,29.0,away,104,2017101600,3277
235643,2017-10-17T03:09:09Z,114.65,-0.04,1.60,0.16,312.13,NaN,2558060.0,Malik Hooker,29.0,away,105,2017101600,3277


In [15]:
## extract only relevant events per play, per safety
events = safety_track_comb.dropna()
events['event'].unique()
key_event = ['pass_forward','pass_arrived']

In [17]:
events.tail(10)

,time,x,y,s,dis,dir,event,nflId,displayName,jerseyNumber,team,frame.id,gameId,playId
235448,2017-10-17T03:08:57Z,67.73,29.36,0.04,0.00,350.03,ball_snap,2556429.0,Matthias Farley,41.0,away,15,2017101600,3277
235486,2017-10-17T03:09:00Z,72.73,27.40,5.49,0.55,104.13,pass_forward,2556429.0,Matthias Farley,41.0,away,53,2017101600,3277
235512,2017-10-17T03:09:03Z,90.01,20.20,7.71,0.79,117.90,pass_arrived,2556429.0,Matthias Farley,41.0,away,79,2017101600,3277
235519,2017-10-17T03:09:04Z,94.74,17.61,6.87,0.74,117.63,pass_outcome_caught,2556429.0,Matthias Farley,41.0,away,86,2017101600,3277
235528,2017-10-17T03:09:05Z,99.58,15.24,4.53,0.48,111.48,touchdown,2556429.0,Matthias Farley,41.0,away,95,2017101600,3277
235553,2017-10-17T03:08:57Z,68.92,18.89,0.02,0.00,214.61,ball_snap,2558060.0,Malik Hooker,29.0,away,15,2017101600,3277
235591,2017-10-17T03:09:00Z,78.91,20.37,6.48,0.68,93.37,pass_forward,2558060.0,Malik Hooker,29.0,away,53,2017101600,3277
235617,2017-10-17T03:09:03Z,96.72,9.36,9.08,0.93,129.46,pass_arrived,2558060.0,Malik Hooker,29.0,away,79,2017101600,3277
235624,2017-10-17T03:09:04Z,101.79,5.39,8.77,0.90,125.75,pass_outcome_caught,2558060.0,Malik Hooker,29.0,away,86,2017101600,3277
235633,2017-10-17T03:09:05Z,108.16,1.40,7.50,0.77,120.52,touchdown,2558060.0,Malik Hooker,29.0,away,95,2017101600,3277


In [14]:

airtime = events[events['event'].isin(key_event)]
airtime.reset_index(inplace=True)
airtime.tail(10)

,index,time,x,y,s,dis,dir,event,nflId,displayName,jerseyNumber,team,frame.id,gameId,playId
2315,212688,2017-10-16T02:54:53Z,72.05,13.93,4.05,0.37,263.72,pass_forward,2552454.0,Landon Collins,21.0,away,45,2017101511,3191
2316,212715,2017-10-16T02:54:56Z,55.03,16.51,7.13,0.74,278.90,pass_arrived,2552454.0,Landon Collins,21.0,away,72,2017101511,3191
2317,212776,2017-10-16T02:54:53Z,70.50,33.74,6.51,0.66,245.12,pass_forward,2555154.0,Darian Thompson,27.0,away,45,2017101511,3191
2318,212803,2017-10-16T02:54:56Z,51.01,22.15,8.63,0.91,246.32,pass_arrived,2555154.0,Darian Thompson,27.0,away,72,2017101511,3191
2319,181821,2017-10-17T02:35:03Z,59.49,20.99,6.25,0.63,165.36,pass_forward,2553822.0,Curtis Riley,35.0,home,59,2017101600,2532
2320,181915,2017-10-17T02:35:03Z,42.24,13.06,3.12,0.34,239.55,pass_forward,2555158.0,Kevin Byard,31.0,home,59,2017101600,2532
2321,235486,2017-10-17T03:09:00Z,72.73,27.40,5.49,0.55,104.13,pass_forward,2556429.0,Matthias Farley,41.0,away,53,2017101600,3277
2322,235512,2017-10-17T03:09:03Z,90.01,20.20,7.71,0.79,117.90,pass_arrived,2556429.0,Matthias Farley,41.0,away,79,2017101600,3277
2323,235591,2017-10-17T03:09:00Z,78.91,20.37,6.48,0.68,93.37,pass_forward,2558060.0,Malik Hooker,29.0,away,53,2017101600,3277
2324,235617,2017-10-17T03:09:03Z,96.72,9.36,9.08,0.93,129.46,pass_arrived,2558060.0,Malik Hooker,29.0,away,79,2017101600,3277


In [33]:

## drop bad data. there should always be a 'pass forward' and 'pass arrived' event in each play
drop_indices = []
for i in range(len(airtime)-1):
    if airtime['event'][i] == airtime['event'][i+1]: #and airtime['playId'][i] != airtime['playId'][i+1]:
        drop_indices.append(i)

print(len(drop_indices))
airtime_filter = airtime.drop(drop_indices,axis=0)
airtime_filter.reset_index(inplace=True)

543


,level_0,index,time,x,y,s,dis,dir,event,nflId,displayName,jerseyNumber,team,frame.id,gameId,playId
0,0,2302,2017-09-08T00:42:45Z,62.65,21.14,7.27,0.76,296.22,pass_forward,496723.0,Eric Berry,29.0,away,49,2017090700,68
1,1,2323,2017-09-08T00:42:47Z,48.48,24.17,5.31,0.58,251.24,pass_arrived,496723.0,Eric Berry,29.0,away,70,2017090700,68
2,2,2650,2017-09-08T00:42:45Z,57.18,23.19,4.87,0.50,260.73,pass_forward,2530794.0,Ron Parker,38.0,away,49,2017090700,68
3,3,2671,2017-09-08T00:42:47Z,46.85,14.94,6.59,0.71,206.67,pass_arrived,2530794.0,Ron Parker,38.0,away,70,2017090700,68
4,7,55847,2017-09-08T01:19:16Z,56.13,19.97,3.66,0.40,286.48,pass_forward,496723.0,Eric Berry,29.0,away,50,2017090700,1025
5,8,55859,2017-09-08T01:19:17Z,52.89,23.68,2.70,0.37,352.17,pass_arrived,496723.0,Eric Berry,29.0,away,62,2017090700,1025
6,9,56108,2017-09-08T01:19:16Z,65.25,37.10,5.69,0.63,263.35,pass_forward,2530794.0,Ron Parker,38.0,away,50,2017090700,1025
7,10,56120,2017-09-08T01:19:17Z,57.84,32.85,6.70,0.77,218.37,pass_arrived,2530794.0,Ron Parker,38.0,away,62,2017090700,1025
8,11,56543,2017-09-08T01:19:16Z,76.98,18.34,2.45,0.27,181.94,pass_forward,2550257.0,Daniel Sorensen,49.0,away,50,2017090700,1025
9,12,56555,2017-09-08T01:19:17Z,75.27,15.30,3.85,0.41,247.58,pass_arrived,2550257.0,Daniel Sorensen,49.0,away,62,2017090700,1025


In [88]:
## convert time string into a datetime type so we can do arithmetic on it
## calculate t2 - t1 and return value in seconds
def time_delta(t1, t2): 
    from datetime import datetime
    tdelta = datetime.strptime(t2.split('T')[1][:-1], '%H:%M:%S') - datetime.strptime(t1.split('T')[1][:-1], '%H:%M:%S')
    return tdelta.seconds

In [118]:
## create a new dataframe detailing the range covered by the safety and time elaped during the play 
range_data = []
for i in range(0,len(airtime_filter)-1,2): 
    ## distance formula for tracking data between the two events
    dist = np.sqrt((airtime_filter['x'][i+1]-airtime_filter['x'][i])**2 + (airtime_filter['y'][i+1]-airtime_filter['y'][i])**2)
    
    ## time elapsed during the play
    tdelta = time_delta(airtime_filter['time'][i], airtime_filter['time'][i+1])
    
    n = airtime_filter['nflId'][i]
    d = airtime_filter['displayName'][i]
    g = airtime_filter['gameId'][i]
    p = airtime_filter['playId'][i]
    #print([n, d, g, p, dist, tdelta])
    range_data.append([n, d, g, p, dist, tdelta])

safety_range = pd.DataFrame(range_data, columns = ['nflId', 'displayName', 'gameId', 'playId', 'rangeCovered', 'playDuration'])

safety_range.tail(10)


,nflId,displayName,gameId,playId,rangeCovered,playDuration
881,2507167.0,Reggie Nelson,2017101510,3400,10.032677,2
882,2555342.0,Karl Joseph,2017101510,3400,11.791963,2
883,2552454.0,Landon Collins,2017101511,1330,2.704958,1
884,2555154.0,Darian Thompson,2017101511,1330,3.514612,1
885,2552454.0,Landon Collins,2017101511,1597,3.447680,2
886,2555154.0,Darian Thompson,2017101511,1597,17.469496,2
887,2552454.0,Landon Collins,2017101511,3191,17.214436,3
888,2555154.0,Darian Thompson,2017101511,3191,22.675718,3
889,2556429.0,Matthias Farley,2017101600,3277,18.720000,3
890,2558060.0,Malik Hooker,2017101600,3277,20.938391,3


In [176]:
deep_pass_track

[                        time      x      y      s   dis     dir      event  \
 2254    2017-09-08T00:42:41Z  74.62  18.19   0.01  0.00  147.47        NaN   
 2255    2017-09-08T00:42:41Z  74.63  18.19   0.01  0.00  151.73        NaN   
 2256    2017-09-08T00:42:41Z  74.63  18.19   0.01  0.00  154.89        NaN   
 2257    2017-09-08T00:42:41Z  74.63  18.19   0.02  0.00  157.14        NaN   
 2258    2017-09-08T00:42:41Z  74.63  18.18   0.02  0.00  158.81        NaN   
 2259    2017-09-08T00:42:41Z  74.63  18.18   0.02  0.00  160.10        NaN   
 2260    2017-09-08T00:42:41Z  74.63  18.18   0.02  0.00  161.42        NaN   
 2261    2017-09-08T00:42:41Z  74.63  18.18   0.03  0.00  163.10        NaN   
 2262    2017-09-08T00:42:41Z  74.63  18.17   0.03  0.00  165.44        NaN   
 2263    2017-09-08T00:42:41Z  74.63  18.17   0.03  0.00  168.65        NaN   
 2264    2017-09-08T00:42:42Z  74.64  18.17   0.03  0.00  173.02        NaN   
 2265    2017-09-08T00:42:42Z  74.64  18.16   0.04  

In [191]:
## to find the target receiver on a play
play_desc = deep_passsort['playDescription'][11913].split(' ')
gameId = deep_passsort['gameId'][11913]
playId = deep_passsort['playId'][11913]
parsed_target = a[a.index('to')+1].split('.',1)

## regex of target receiver to match against 'displayName'
import re
regex = '$' + re.escape(parsed_target[0]) + '.* ' + re.escape(parsed_target[1])
regex

i = 22
b = deep_pass_track[i][deep_pass_track[i]['displayName']]
print(b)

target_tracking = deep_pass_track.copy()
#for i in range(len(deep_pass_track)):
    #dpt.tail(10)
    #a = dpt.loc[deep_pass_track['displayName'].contains(parsed_target[1])]
    #print(a)
 #   target_tracking[i] = deep_pass_track[i][deep_pass_track[i]['displayName'].contains(parsed_target[1])]
#target_tracking    


KeyError: "['Kyle Rudolph' 'Kyle Rudolph' 'Kyle Rudolph' ... 'football' 'football'\n 'football'] not in index"